In [140]:
#libraries needed for this code run
!pip install bs4
!pip install pandas
!pip install requests
!pip install numpy

In [141]:
#imports libraries
#beautiful soup for pulling data out of HTML and XML files
from bs4 import BeautifulSoup
# requests to use a get request
import requests
#pandas to store data in a dataframe
import pandas as pd
#numpy to deal with nan data
import numpy as np

In [149]:
# Function to extract Product Title
def get_title(soup):
    try:
        # Outer Tag Object
        title = new_soup.find("h1",attrs={'class':'title-1'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.split('\n')[1].strip()+title_value.split('\n')[3].strip()
        
    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Original price
def get_OriginalPrice(soup):
    try:
        
        OriginalPrice = new_soup.find("span",attrs={'class':'body-1-light-striked flex items-end mr-2 cursor-pointer'}).text.strip()

    except AttributeError:
        OriginalPrice = "Not Available"	

    return OriginalPrice    

# Function to extract Current Product Price
def get_price(soup):

    try:
        price = new_soup.find("div",attrs={'class':'text-primary'}).text.strip()

    except AttributeError:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = new_soup.find("span",attrs={'class':"ml-1 text-primary body-2-bold"}).text.strip()
    
    except AttributeError:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'class':'ml-2 body-2-bold underline _1HL64gTNKr5pbdUa5n9_1Y'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count




In [150]:
if __name__ == '__main__':

    # add your user agent 
    Headers = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    })

    # The webpage URL
    URL = "https://www.backmarket.nl/nl-nl/l/iphone/aabc736a-cb66-4ac0-a3b7-0f449781ed39?gclid=Cj0KCQjwi46iBhDyARIsAE3nVral_leWXn8KOG6hlma7u3b1tTQ2Eg6-xAYr9YJHhQsKzaQELmiwEd8aAuzaEALw_wcB"
    
    # HTTP Request
    webpage = requests.get(URL, headers=Headers)
    
    #status code 200 means our request was accepted
    print(webpage.status_code)
    
    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # Fetch links as List of Tag Objects
    links = soup.find_all("a",attrs={'class':'focus:outline-none group md:box-border relative'})

    # Store the links
    links_list = []
        # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "Original Price":[],"Current Price":[] ,"rating":[], "reviews":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.backmarket.nl" + link, headers=Headers)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['Original Price'].append(get_OriginalPrice(new_soup))
        d['Current Price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))

    
        amazon_df = pd.DataFrame.from_dict(d)
        amazon_df['title'].replace('', np.nan, inplace=True)
        amazon_df = amazon_df.dropna(subset=['title'])
        amazon_df.to_csv("amazon_data.csv", header=True, index=False)

200
